In [ ]:
# !pip install yt-dlp
# !pip install opencv-python
# !pip install pandas
# !pip install google-colab
# !pip install tensorflow

In [1]:
import os
import json
import yt_dlp
import cv2
import math
import random
import concurrent.futures
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from tensorflow.keras.utils import to_categorical

In [5]:
def download_and_extract_frames(video_data, save_frames_path):
    video_url = video_data['url']
    # ISSUE 2: using the video id is not good because it is using file, which is not the same as the words. Also, it is not ideal because two words can have the same file. Two solutions here:
    # 1. Create a folder with the label then inside that folder, we have laugh 1, laugh 2, laugh 3, etc ---> trying this
    # 2. Create a csv or json to hold the label and the link 
    video_id = video_data['file']
    label = video_data['clean_text']
    start_time = video_data['start_time']
    end_time = video_data['end_time']
    output_filename = label + " " + video_id
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]',
        'quiet': True,
        'no_warnings': True
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            video_info = ydl.extract_info(video_url, download=False)
            video_url = video_info['url']
    except Exception as e:
        print(f"Error downloading {video_url}: {e}")
        return

    print(f"Video extracted successfully to {output_filename}")

    ################################## ISSUE 1: It is breaking whole video into frames, not just a clip
    ################################## SOLUTION:
    cap = cv2.VideoCapture(video_url)
    label_dir = os.path.join(save_frames_path, label)
    output_dir = os.path.join(label_dir, video_id)
    #Create a folder for a label
    os.makedirs(label_dir, exist_ok=True)
    #Create the folder to save the frames within the label
    os.makedirs(output_dir, exist_ok= True)

    # Get frames per second and total number of frames
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame indices based on start and end timestamps
    start_frame_index = int(start_time * fps)
    end_frame_index = int(end_time * fps)

    # Set the capture position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_index)

    # Read frames between start and end timestamps
    # frames = []
    frame_idx = start_frame_index
    while frame_idx <= end_frame_index:
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_dir, f"frame_{frame_idx}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_idx += 1

    # Release the video capture object
    cap.release()
    ############################### END OF SOLUTION 
   
# checkpoint_path = "/content/drive/My Drive/Colab Notebooks/MS-ASL/preprocessing_checkpoint.txt"
#print(os.path.exists(checkpoint_path))
#print(os.listdir("/content/drive/My Drive/Colab Notebooks/MS-ASL/"))

def preprocess_data_multithreaded(data, save_frames_path, checkpoint_path, max_workers=10):
    # Load existing checkpoints if available
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as file:
            processed_videos = set(line.strip() for line in file.readlines())
    else:
        processed_videos = set()

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Filter out already processed videos and videos without a 'file' key
        filtered_data = [item for item in data if 'file' in item and item['file'] not in processed_videos]

        # Process videos and update checkpoint after each video
        for item in filtered_data:
            download_and_extract_frames(item, save_frames_path)

            # Debugging information
            print(f"Processed video: {item['file']}")

            # os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
            with open(checkpoint_path, 'a') as file:
                file.write(item['file'] + '\n')

            # Debugging information
            print(f"Updated checkpoint with video: {item['file']}")




In [53]:

# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

with open('MSASL_train.json', 'r') as file:
  train_data = json.load(file)[:1000]

with open('MSASL_val.json', 'r') as file:
  val_data = json.load(file)[:1000]
    
# filtered_data = [obj for obj in train_data if obj.get('clean_text') == desired_value]
#getting 30 randoms words from train
# random_lines = random.sample(train_data, 30)
# clean_texts = [obj.get('clean_text') for obj in random_lines]
# for line in clean_texts:
#     print(line)
# print(len(clean_texts))

#get all the matching words between train and validation
matching_rows = [row1 for row1 in train_data for row2 in val_data if row1.get('clean_text') == row2.get('clean_text')]
my_set = set()
for line in matching_rows:
    my_set.add(line['clean_text'])

random_words = random.sample(list(my_set), 30)
print(random_words)

#making sure validation set has all these words (skip)
#get all the rows from train and validation sets that have the matching values
# val_data = [obj for obj in val_data if all( obj.get('clean_text') in clean_texts)]
# bool = all(obj.get('clean_text') in random_words for obj in val_data)
matching_train = [row for row in train_data if row.get('clean_text') in random_words]
matching_val = [row for row in val_data if row.get('clean_text') in random_words]

print(len(matching_train))
print(len(matching_val))

# print(matching_train)
# print(matching_val[:10])
# print("\n")
# print(val_data[:10])

Video extracted successfully to sign language SIGN-LANGUAGE-S-CLAW-F
Processed video: SIGN-LANGUAGE-S-CLAW-F
Updated checkpoint with video: SIGN-LANGUAGE-S-CLAW-F


ERROR: [youtube] 1AyT77LqJzQ: Video unavailable


Error downloading https://www.youtube.com/watch?v=1AyT77LqJzQ: ERROR: [youtube] 1AyT77LqJzQ: Video unavailable
Processed video: ASL Vocabulary school
Updated checkpoint with video: ASL Vocabulary school


ERROR: [youtube] 1AyT77LqJzQ: Video unavailable


Error downloading https://www.youtube.com/watch?v=1AyT77LqJzQ: ERROR: [youtube] 1AyT77LqJzQ: Video unavailable
Processed video: ASL Vocabulary school
Updated checkpoint with video: ASL Vocabulary school
Video extracted successfully to easter Easter
Processed video: Easter
Updated checkpoint with video: Easter
Video extracted successfully to boring ASL Boring 
Processed video: ASL Boring 
Updated checkpoint with video: ASL Boring 
Video extracted successfully to letter LETTER(3)
Processed video: LETTER(3)
Updated checkpoint with video: LETTER(3)
Video extracted successfully to shock shock-BRAIN-FREEZE
Processed video: shock-BRAIN-FREEZE
Updated checkpoint with video: shock-BRAIN-FREEZE
Video extracted successfully to breakfast ASL Breakfast  
Processed video: ASL Breakfast  
Updated checkpoint with video: ASL Breakfast  
Video extracted successfully to clock clock(2)
Processed video: clock(2)
Updated checkpoint with video: clock(2)
Video extracted successfully to important essential-IMP

...

Error downloading https://www.youtube.com/watch?v=lAE4m0AFNcw: ERROR: [youtube] lAE4m0AFNcw: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
Processed video: ASL Happy
Updated checkpoint with video: ASL Happy
Video extracted successfully to jealous JEALOUS(2)
Processed video: JEALOUS(2)
Updated checkpoint with video: JEALOUS(2)


ERROR: [youtube] 0DcZj6ifH4I: Private video. Sign in if you've been granted access to this video


Error downloading www.youtube.com/watch?v=0DcZj6ifH4I: ERROR: [youtube] 0DcZj6ifH4I: Private video. Sign in if you've been granted access to this video
Processed video: SignSchool Long(1)
Updated checkpoint with video: SignSchool Long(1)


ERROR: [youtube] vegwSPDHPEk: Private video. Sign in if you've been granted access to this video


Error downloading www.youtube.com/watch?v=vegwSPDHPEk: ERROR: [youtube] vegwSPDHPEk: Private video. Sign in if you've been granted access to this video
Processed video: SignSchool Broom Sweep
Updated checkpoint with video: SignSchool Broom Sweep
Video extracted successfully to shock SHOCKED(1)
Processed video: SHOCKED(1)
Updated checkpoint with video: SHOCKED(1)
Video extracted successfully to disappointed DISAPPOINT
Processed video: DISAPPOINT
Updated checkpoint with video: DISAPPOINT


In [92]:
# Processing new ASL datasets
import shutil

#source_path
source_folder = 'newasl/sign-language-gesture-recognition/all_videos/'
train_folder = 'newasl/sign-language-gesture-recognition/train_videos/'
test_folder = 'newasl/sign-language-gesture-recognition/test_videos/'

# Get a list of all labels in train_folder
subfolders = [subfolder for subfolder in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, subfolder))]

# For each label
for subfolder_name in subfolders:
    #Get the path
    train_subfolder_path = os.path.join(train_folder, subfolder_name)
    #Get all the videos 
    videos = [subfolder for subfolder in os.listdir(train_subfolder_path) if os.path.isfile(os.path.join(train_subfolder_path, subfolder)) and subfolder.endswith(".mp4")]

    #For each video
    for video in videos:
        
        #Get the video path
        old_video_path = os.path.join(train_subfolder_path, video)
        print(old_video_path)
        # print(video)
        # print(video[6])
        string_number = video[6]
        # print(string_number)
        if(string_number == "0"):
            new_folder_name = video[:5] + "08" + video[7:]
            print(new_folder_name)
        else:
            integer_number = int(string_number) -2
            # print(integer_number)
            new_folder_name = video[:6] + str(integer_number) + video[7:]
            # print(new_folder_name)
        
        new_folder_path = os.path.join(train_subfolder_path, new_folder_name)
        os.rename(old_video_path, new_folder_path)
        # prefix = video[4:7]
        # # print(prefix)
        # if prefix == '001' or prefix == '002':
        #     shutil.move(video_path, test_subfolder_path)
        # else:
        #     shutil.move(video_path, train_subfolder_path)

# subfolders = [subfolder for subfolder in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, subfolder))]


newasl/sign-language-gesture-recognition/train_videos/Accept\050_003_001.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_003_002.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_003_003.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_003_004.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_003_005.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_004_001.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_004_002.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_004_003.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_004_004.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_004_005.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_005_001.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_005_002.mp4
newasl/sign-language-gesture-recognition/train_videos/Accept\050_005_003.mp4

In [59]:
train_labels = [subfolder for subfolder in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, subfolder))]
test_labels = [subfolder for subfolder in os.listdir(test_folder ) if os.path.isdir(os.path.join(test_folder, subfolder))]

print(train_labels)
print(test_labels)

print(len(train_labels))

['africa', 'ball', 'bath', 'before', 'black', 'buy', 'chat', 'cook', 'dry', 'eat', 'fine', 'forget', 'gold', 'green', 'happy', 'holland', 'how_many', 'i', 'important', 'jacket', 'learn', 'mustache', 'not', 'now', 'pizza', 'play', 'school', 'shock', 'shoes', 'so', 'table', 'yes']
['africa', 'ball', 'bath', 'before', 'black', 'buy', 'chat', 'cook', 'dry', 'eat', 'fine', 'forget', 'gold', 'green', 'happy', 'holland', 'how_many', 'i', 'important', 'jacket', 'learn', 'mustache', 'not', 'now', 'pizza', 'play', 'school', 'shock', 'shoes', 'so', 'table', 'yes']
32


In [62]:
# import os
# ! pip install sklearn
import numpy as np
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, Input
# from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Paths to the downloaded and extracted datasets
# asl_alphabet_dataset_path = '/path/to/asl-alphabet'
# sign_language_mnist_path = '/path/to/sign-language-mnist'
TRAIN_FRAMES_PATH = 'train_chosen'
VAL_FRAMES_PATH = 'val_chosen'

# Model parameters
num_frames_per_sequence = 5  # For ASL alphabet, you might use single frames
input_shape = (224, 224, 3)
batch_size = 32
epochs = 10

# Assuming the structure of the datasets is such that each class/letter is in a separate folder
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    # validation_split=0.2  # Assuming you want to split the dataset
)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_FRAMES_PATH,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    # subset='training'
)

train_classes = train_generator.class_indices
# validation_classes = validation_generator.class_indices

validation_generator = validation_datagen.flow_from_directory(
    VAL_FRAMES_PATH,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    # classes=train_classes.keys() 
    # subset='validation' #this doesn't work for some reasons
)

# Build model function/ orignal Yashi model
def build_model(input_shape, num_classes):
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg16.trainable = False
    model = Sequential()
    model.add(vgg16)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model
    
# # # Build model function/ model with 2 drop out and LSTM
# def build_model(input_shape, num_classes):
#     vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape[1:])
#     for layer in vgg16.layers:
#         layer.trainable = False
#     model = Sequential()
#     model.add(TimeDistributed(vgg16))
#     model.add(TimeDistributed(Flatten()))
#     model.add(Dropout(0.5))
#     model.add(LSTM(512, return_sequences=False))
#     model.add(Dropout(0.3))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(num_classes, activation='softmax'))
#     return model
    
# Build and compile the model
num_classes = train_generator.num_classes
print(num_classes)
model = build_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('model_asl.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stopping]
)


Found 13135 images belonging to 32 classes.
Found 1580 images belonging to 32 classes.
32
Epoch 1/10
410/410 [==============================] - ETA: 0s - loss: 2.7447 - accuracy: 0.2416
Epoch 1: val_accuracy improved from -inf to 0.06059, saving model to model_asl.h5


C:\Users\dangk\AppData\Local\Programs\Python\Python36\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


410/410 [==============================] - 2688s 7s/step - loss: 2.7447 - accuracy: 0.2416 - val_loss: 4.4165 - val_accuracy: 0.0606
Epoch 2/10
410/410 [==============================] - ETA: 0s - loss: 1.9256 - accuracy: 0.4012
Epoch 2: val_accuracy did not improve from 0.06059
410/410 [==============================] - 2683s 7s/step - loss: 1.9256 - accuracy: 0.4012 - val_loss: 5.3263 - val_accuracy: 0.0548
Epoch 3/10
410/410 [==============================] - ETA: 0s - loss: 1.6000 - accuracy: 0.4872
Epoch 3: val_accuracy did not improve from 0.06059
410/410 [==============================] - 2675s 7s/step - loss: 1.6000 - accuracy: 0.4872 - val_loss: 5.4007 - val_accuracy: 0.0606
Epoch 4/10
410/410 [==============================] - ETA: 0s - loss: 1.4404 - accuracy: 0.5288
Epoch 4: val_accuracy did not improve from 0.06059
410/410 [==============================] - 2619s 6s/step - loss: 1.4404 - accuracy: 0.5288 - val_loss: 6.7633 - val_accuracy: 0.0555
Epoch 5/10
410/410 [=======